In [ ]:
import numpy as np
import types
import argparse
import matplotlib.pyplot as plt
import time
import os
import sys
import copy
import torch
import torch.nn as nn
list_pathstoadd = [
       "../../"
]
for path in list_pathstoadd:
    if(path not in sys.path):
        sys.path.append(path)
#import generalGPmodule
import gpex
import gpex.kernelmappings.image
import skimage
from skimage import io
from sklearn.metrics import confusion_matrix
#import torchofgp.tgp as tgp
from gpex.kernelmappings.image import Resnet50BackboneKernelDivideAvgPool
#from hpresnet18 import PrimaryNetwork
#import relatedwork
#from relatedwork.utils.generativemodels import ResidualEncoder
import akresnetformnist
from akresnetformnist import *
import loadmnist_fromsplit
from loadmnist_fromsplit import *


In [ ]:
#settings ================
#fname_split = args.fname_split #"split_0.pkl"
#tune for each run ==
#numepochs_1, numepochs_2 = args.numepochs_1, args.numepochs_2 #10000, 10 #---overrideint(sys.argv[1]), int(sys.argv[2])
#countupdateU_afterupdateGP = 10 #---override int(sys.argv[3]) #should be 100
#idx_trainingbatch = 1
#flag_enabledataaugmentation = True

#str_versionname = "nbrun" #---overrid"explainann_"+str(sys.argv[-1])
#int_mode_modulekernel, num_backbones = 16, None
du_per_class = 20
rng_randw = [-1.0, 2.0]
#flag_train_memefficient, memefficeint_heads_in_compgraph = False, None
dim_wideoutput = 1024
#stepsize_gradupdate = 1
#tune for each dataset
batchsize = 8
num_classes = 10
#general settings ===
#idx_split = 0
#int_exposedclass = None

#flag_efficient = True
#flag_detachcovpvn = True
#flag_controlvariate = True
#flag_setcovtoOne = False
#int_mode_controlvariate = 2
#val_clipgradient = None #set to None to avoid any gradient clipping.
#fname_phase1output = None#"W0-1_relatively_fit/4872" #"model_0.96_outputwidth_1024"
#mode_ann = 1
rootpath_ds = "./NonGit/MNIST/"
#"./cifar-10-batches-py"
#if(mode_ann==1):
#fname_phase0output = args.fname_phase0output
#"Phase0_DifferentSplits/checkpoint/split_0/ckpt_200_88.22.pth"
#"model_0.9538_outputwidth_1024"
#elif(mode_ann==2):
#    assert False
#    fname_phase0output = "model_0.9538_outputwidth_1024"
#"model_0.9814024796693774_outputwidth_1024_gpmodelinitialized"
#assert((fname_phase1output is None) or (fname_phase0output is None))
#assert(int_mode_modulekernel in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17])
# if(num_backbones is not None):
#     assert(int_mode_modulekernel in [6,7])



In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
print("Training set:")
ds_train = loadmnist_fromsplit.MNISTDatasetFromSplit(
    rootdir = rootpath_ds,
    fname_split = "split_0.pkl",
    str_trainortest = "train",
    flag_enabledataaugmentation = False,
    rootpath_splits = "./Splits/"
)
print("Recurring:")
ds_recurring = loadmnist_fromsplit.MNISTDatasetFromSplit(
    rootdir = rootpath_ds,
    fname_split = 'split_0.pkl',
    str_trainortest = "train",
    flag_enabledataaugmentation = False,
    rootpath_splits = "./Splits/"
)
print("Test:")
ds_test = loadmnist_fromsplit.MNISTDatasetFromSplit(
    rootdir = rootpath_ds,
    fname_split = 'split_0.pkl',
    str_trainortest = "test",
    flag_enabledataaugmentation = False,
    rootpath_splits = "./Splits/"
)



In [ ]:
dl_train = DataLoader(ds_train, batch_size=batchsize,
                      shuffle=True, num_workers=2)
dl_recurring = DataLoader(ds_recurring, batch_size=batchsize,
                          shuffle=False, num_workers=2)
dl_test = DataLoader(ds_test, batch_size=batchsize,
                     shuffle=False, num_workers=0)

In [ ]:
class ClampAndTanh(torch.nn.Module):
    def __init_(self, minval=-1.0, maxval=1.0):
        self.minval, self.maxval = minval, maxval
        super(ClampAndTanh, self).__init__()
        
        
    def forward(self, x):
        output = torch.nn.functional.tanh(
                        torch.clamp(
                            x, -1.0, 1.0
                        )
                    )
        return output
    
def initweights_to_zero(m):
    if type(m) in {nn.Linear, nn.Conv2d, nn.Linear}:
        torch.nn.init.zeros_(m.weight)
        m.bias.data.fill_(np.random.randn()*0.1) #TODO:check

In [ ]:
class ModuleF1(torch.nn.Module):
    def __init__(self, module_caller):
        super(ModuleF1, self).__init__()
        #make internals ===
        self.module = Resnet50BackboneKernelDivideAvgPool(
            num_classes = num_classes,
            du_per_class = du_per_class
        )
    
    def set_rng_outputheads(self, rng_outputhead):
        self.module.set_rng_outputheads(rng_outputhead)
    
    
    def forward(self, x):
        toret = self.module(x)
        toret = toret.unsqueeze(-1).unsqueeze(-1)
        return toret

In [ ]:
class MainModule(nn.Module):
    def __init__(self, num_classes, device, dl_recurring, dl_nonrecurring, dl_test, batchsize, dim_wideoutput):
        '''
        Inputs:
            - size_input: size of the input, e.g., [32 x 2000 x 7 x 7].
            - device: the device on which the GP fields are going to be created.
            - num_outputheads: an integer, number of output heads.
        '''
        super(MainModule, self).__init__()
        #grab args ===
        self.num_classes = num_classes
        self.device = device
        self.dl_recurring = dl_recurring
        self.dl_nonrecurring = dl_nonrecurring
        self.dl_test = dl_test
        self.batchsize = batchsize
        #self.iter_dl_recurring = iter(self.dl_recurring)
        #make internal module_tobecomeGP ===
        self.module_tobecomeGP = AkResNet18(
            num_classes=num_classes,
            dim_wideoutput=dim_wideoutput
        )
        #make internals ===
        self.dic_dlname_to_iter = {
            "dl_recurring":iter(self.dl_recurring),
            "dl_nonrecurring":iter(self.dl_nonrecurring),
            "dl_test":iter(self.dl_test)
        }
        #make module f1 ===
        self.module_f1 = ModuleF1(self) #nn.Sequential(
        self._lastidx_recurring = []
        #internal field to subsample when feeding minbatch ====
        self.n_subsampleminibatch = None
            
    def forward(self, x, y, n):
         return self.module_tobecomeGP(x), y, n
        
    
    def func_get_modulef1(self):
        return self.module_f1
    
    def func_mainmodule_to_moduletobecomeGP(self, module_input):
        return module_input.module_tobecomeGP
    
    
    def _func_feed_minibatch(self, dl_input, str_dlname, flag_addnoisetoX = False):
        #print("reached here 1")
        if(False):#iter_dl is None):
            pass #x, y, n = next(iter(dl_input))
        else:
            try:
                x, y, n = next(self.dic_dlname_to_iter[str_dlname])
            except (StopIteration):
                self.dic_dlname_to_iter[str_dlname] = iter(dl_input)
                x, y, n = next(self.dic_dlname_to_iter[str_dlname])
                
        
        if(flag_addnoisetoX == True):
            idx_permutex = np.random.permutation([u for u in range(list(x.size())[0])]).tolist()
            idx_permutex = torch.LongTensor(idx_permutex)
            x_perumted = x[idx_permutex, :, :, :]
            
            rand_w = torch.rand((list(x.size())[0]))                        .float().unsqueeze(-1).unsqueeze(-1).unsqueeze(-1) #[Nx1x1x1]
            rand_w = rand_w*(rng_randw[1]-rng_randw[0]) + rng_randw[0] #in [rng[0] , rng[1]]
            rand_w = rand_w.detach()
            #print("rand_w.shape = {}".format(rand_w.shape))
            x = rand_w*x + (1.0-rand_w)*x_perumted  #[N x 3 x 224 x 224]
            x = x + 0.1*torch.randn_like(x).float()
        
        if(self.n_subsampleminibatch is None):
            pass
        else:
            x = x[0:self.n_subsampleminibatch, :,:,:]
            y = y[0:self.n_subsampleminibatch]
            n = n[0:self.n_subsampleminibatch]
        
        if(dl_input.dataset == ds_recurring):
            self._lastidx_recurring = n
            if(len(n) != list(x.size())[0]):
                assert False
        
        output, _, _ = self.forward(x.to(self.device), y, n)
        
        return output, y, n
    
    def func_feed_recurring_minibatch(self):
        output, y, n = self._func_feed_minibatch(
                            self.dl_recurring,
                            str_dlname = "dl_recurring"
                         )
        return output, y, n
    
    def func_feed_noise_minibatch(self):
        output, y, n = self._func_feed_minibatch(
                        self.dl_nonrecurring,
                        str_dlname = "dl_nonrecurring",
                        flag_addnoisetoX=True
                      )
        return output, y, n
    
    def func_feed_nonrecurring_minibatch(self):
        output, y, n = self._func_feed_minibatch(
                                self.dl_nonrecurring,
                                str_dlname = "dl_nonrecurring",
                                flag_addnoisetoX=False
                              )
        return output, y, n
    
    def func_feed_test_minibatch(self):
        output, y, n = self._func_feed_minibatch(
                                    self.dl_test,
                                    str_dlname = "dl_test"
                                )
        return output, y, n
    
    def func_get_indices_lastrecurringinstances(self):
        return self._lastidx_recurring.cpu().numpy().tolist()



In [ ]:
model = MainModule(
    dim_wideoutput=dim_wideoutput,
    num_classes = num_classes,
    device = device,
    dl_recurring = dl_recurring,
    dl_nonrecurring = dl_train,
    dl_test = dl_test,
    batchsize = batchsize
)
state_dict = torch.load(
        "NonGit/checkpoint_train_split0.pth"
)
model.module_tobecomeGP.load_state_dict(
       state_dict,
       strict = True
    )
print("classifier was loaded from checkpoint.")

In [ ]:
#let's compute the accuracy of the classifier on the test set.
def evaluate_classifier(model_input, ds_input, input_device):
    model_input.eval()
    with torch.no_grad():
        toret = []
        list_gty = []
        for n in range(len(ds_input)):
            print(" instance {} from {}".format(n, len(ds_input)), end='\r')
            x, y, _ = ds_input[n]
            output = model_input(x.unsqueeze(0).to(input_device))
            toret.append(output[0,:].detach().cpu().numpy())
            list_gty.append(y)
        print("\n")
        toret = np.array(toret) #[N x 10]
        #toret = toret[:,0,:]
    model_input.train()
    return toret, list_gty
predy, list_gty = evaluate_classifier(model.module_tobecomeGP, ds_test, device)
np_confmatrix = confusion_matrix(np.argmax(predy,1), list_gty)
acc = np.sum(np_confmatrix*np.eye(10))/np.sum(np_confmatrix)
print(acc)

In [ ]:
model.to(device)
#model.n_subsampleminibatch = 50
gpmodel = gpex.GPEXModule(
    module_rawmodule = model,
    size_recurringdataset = len(ds_recurring),
    device = device,
    func_mainmodule_to_moduletobecomeGP = model.func_mainmodule_to_moduletobecomeGP, 
    func_feed_noise_minibatch = model.func_feed_noise_minibatch,
    func_feed_inducing_minibatch  = model.func_feed_recurring_minibatch,
    func_feed_nonrecurring_minibatch = model.func_feed_nonrecurring_minibatch,
    func_feed_test_minibatch = model.func_feed_test_minibatch,
    func_get_indices_lastrecurringinstances = model.func_get_indices_lastrecurringinstances,
    func_get_modulef1 = model.func_get_modulef1,
    flag_efficient = True,
    flag_detachcovpvn = True,
    flag_controlvariate = True,
    flag_setcovtoOne = False,
    int_mode_controlvariate = 2,
    flag_train_memefficient = False,
    memefficeint_heads_in_compgraph = None
  )
#model.n_subsampleminibatch = 50
gpmodel.sigma2_GP = 1.0 #TODO:check
gpmodel.train()
gpmodel.to(device)
print("gpmodel was created on {}".format(device))


In [ ]:
#initialize the gpmodel
gpmodel.init_UV()
print("Initialized U&V from module f1(.).")
gpmodel.train()
initweights_to_zero(gpmodel.module_f1)
print("Weights were initialized to zero.")

In [ ]:
# set_params_gpwithoutuncertainty = set(list(gpmodel.parameters())) - set(gpmodel.func_mainmodule_to_moduletobecomeGP(model).parameters()) 
# optimizer_straight = torch.optim.Adam(
#         list(set_params_gpwithoutuncertainty), lr=0.0001, amsgrad=True
#     )
# optimizer_g = torch.optim.Adam(
#         list(gpmodel.module_tobecomeGP.parameters())+\
#         [], lr=0.00001, amsgrad=True
#     )#TODO:check
# optimizer_GPY = torch.optim.Adam([gpmodel.GP_Y], lr=0.00001) #TODO:check
#set_modelparams = gpmodel.module_f1.parameters()
optimizer_modelparams = torch.optim.Adam(
                 gpmodel.module_f1.parameters(),
                lr=0.0001, amsgrad=False
            )
# scheduler_modelparams = torch.optim.lr_scheduler.CosineAnnealingLR(
#     optimizer_modelparams, T_max=200
# )
# list(model.module_gp.module_fullyconvpart.parameters())+\
#                 [model.module_gp.GP_Y]+\
#                 list(model.module_fullyconnected.parameters())
# criterion = torch.nn.CrossEntropyLoss()

In [ ]:
# #set path_output_dump ===
# if(flag_isrunning_slurm == False):
#     path_output_dump = "NonGit/{}".format(args.fname_split[0:-4])
# else:
#      path_output_dump = "/home/ah8uofa/scratch/Rebuttal_Neurips2023/{}/{}".format(
#            os.getcwd().split('/')[-1],
#          args.fname_split[0:-4]
#      )
# print("path_output_dump is set to {}.".format(path_output_dump))

In [ ]:
#%%capture
if('history_Qvn_loss' not in globals()):
    history_modelparamsloss = []
    #history_Qvn_loss, history_Qvhatm_loss, history_modelparamsloss = [], [], []
    #history_Qvn_loss_celoss, history_Qvn_loss_klterm = [], []
    #history_Qvhatm_loss_dot_term1, history_Qvhatm_loss_dot_kl = [], []
    #history_celoss = []
    #history_straightceloss = []
    batchcount = 0
    #count_updatevarpars = 0
    #flag_GPXinitialized = False
    
    #torch.save(
    #    gpmodel.state_dict(),
    #    "{}/CheckpointAvoidNan/gpmodel_phase2_w{}.pt".format(path_output_dump, dim_wideoutput)
    #)
    #backup_optimizer_straight = copy.deepcopy(optimizer_straight)
    #backup_optimizer_g = copy.deepcopy(optimizer_g)
    #backup_optimizer_GPY = copy.deepcopy(optimizer_GPY)
    #backup_optimizer_modelparams = copy.deepcopy(optimizer_modelparams)
    #count_updatestomodelparams = 0
    
#count_dlrecurringvisited = 0
#count_backward_qvn = 1
#count_backward_qvhatm = 1
#count_backward_modelparams = 1

#zero-fill all gradients ===
#optimizer_straight.zero_grad()
#optimizer_g.zero_grad()
#optimizer_GPY.zero_grad()
#optimizer_modelparams.zero_grad()


t_beforeepoch_1 = time.time()
for epoch in range(2):
    
    for idx, data in enumerate(dl_train):
        batchcount += 1
        if((batchcount%100) == 0):
            print("batchcount = {}".format(batchcount))
        
        if(((batchcount%1000) == 0) or (batchcount==0)):    
            gpmodel.renew_precomputed_XTX()
            print("XTX renewed (to avoid propagating numerical error.)")
        
        
        #if(True):
        #    if((count_updatevarpars%1) == 0):#TODO:change
        #count_backward_modelparams += 1
        #model.n_subsampleminibatch = 15
        optimizer_modelparams.zero_grad()

        cost_modelparams = gpmodel.getcost_GPmatchNN()
        if(gpmodel.flag_svdfailed == True):
            pass #in this case the svd has failed.
            assert False #TODO:delete
        else:


            #netout_modelparams, y_modelparams, _ = output_modelparams

            #cost_modelparams = costmodelparams_term1 #/(costmodelparams_term1.detach())
            #if(flag_train_memefficient == False):
            #cost_modelparams = cost_modelparams/(stepsize_gradupdate+0.0)
            cost_modelparams.backward()
            optimizer_modelparams.step()

            #print("    performed the backprop 3.")
            #if((count_backward_modelparams%stepsize_gradupdate) == 0):
                #pass
                #optimizer_modelparams.step()
                #optimizer_modelparams.zero_grad()
                #print("   >>> backprop3.step.")

            #count_updatestomodelparams += 1
            history_modelparamsloss.append(cost_modelparams.detach().cpu().numpy().tolist())
            #history_celoss.append(celoss_modelprams.detach().cpu().numpy().tolist())



            #update GPX ====
            gpmodel.update_U()
            #print("         performed the update to GPX.")
                    


In [ ]:
plt.figure()
plt.plot(range(len(history_modelparamsloss)), history_modelparamsloss)
plt.show()

In [ ]:
#check if g(.) and GP path match on test instances ====
gpmodel.renew_precomputed_XTX()
list_outputgpoutputg = gpmodel.checkequal_GPout_ANNout_ontest(10)
for n in range(len(list_outputgpoutputg)):
    a = list_outputgpoutputg[n][0]
    b = list_outputgpoutputg[n][1]


    min_ab = min(np.min(a), np.min(b))
    max_ab = max(np.max(a), np.max(b))

    plt.figure(figsize=(6,6))
    plt.subplot(1,2,1)
    plt.imshow(np.round(a, 2), vmin=min_ab, vmax=max_ab, aspect="auto"); plt.colorbar()
    plt.title("GPs output")
    plt.subplot(1,2,2)
    plt.imshow(np.round(b, 2), vmin=min_ab, vmax=max_ab, aspect="auto"); plt.colorbar()
    plt.title("ANN output")
    plt.show()

In [ ]:
# def evaluate_model(model_input, ds_input, input_device):
#     model_input.eval()
#     with torch.no_grad():
#         toret = []
#         list_gty = []
#         for n in range(len(ds_input)):
#             print(" instance {} from {}".format(n, len(ds_input)), end='\r')
#             x, y, _ = ds_input[n]
#             output = model_input(x.unsqueeze(0).to(input_device))
#             toret.append(output[0,:].detach().cpu().numpy())
#             list_gty.append(y)
#         print("\n")
#         toret = np.array(toret) #[N x 10]
#         #toret = toret[:,0,:]
#     model_input.train()
#     return toret, list_gty

In [ ]:
if(flag_isrunning_jupyterNB):
    predy, list_gty = evaluate_model(model.module_tobecomeGP, ds_test, device)
    np_confmatrix = confusion_matrix(np.argmax(predy,1), list_gty)
    #print(np_confmatrix)
    #print("\n\n\n")
    acc = np.sum(np_confmatrix*np.eye(10))/np.sum(np_confmatrix)
    print(acc)

In [ ]:
count_dlrecurringvisited = 0
count_backward_qvn = 1
count_backward_qvhatm = 1
count_backward_modelparams = 1

#zero-fill all gradients ===
optimizer_straight.zero_grad()
optimizer_g.zero_grad()
optimizer_GPY.zero_grad()
optimizer_modelparams.zero_grad()


t_beforeepoch_2 = time.time()
for epoch in range(2):
    
    if(epoch>0):
        torch.save(
            gpmodel.state_dict(),
            "{}/round2_epoch_{}.pt".format(path_output_dump, epoch)
        )
    
    for idx, data in enumerate(dl_train):
        batchcount += 1
        if((batchcount%100) == 0):
            print("batchcount = {}".format(batchcount))
        if(((batchcount%1000) == 0) or (batchcount==0)):    
            gpmodel.renew_precomputed_XTX()
            print("XTX renewed (to avoid propagating numerical error.)")
        
        
        if(True):
            if((count_updatevarpars%1) == 0):#TODO:change
                count_backward_modelparams += 1
                #model.n_subsampleminibatch = 15
                
                costmodelparams_term1 = gpmodel.getcost_GPmatchNN()
                if(gpmodel.flag_svdfailed == True):
                    pass #in this case the svd has failed.
                else:
                    
                    
                    #netout_modelparams, y_modelparams, _ = output_modelparams
                    if(int_exposedclass is None):
                        pass#celoss_modelprams = criterion(netout_modelparams, y_modelparams.to(device))
                    else:
                        pass#celoss_modelprams = torch.Tensor([0.0])
                    cost_modelparams = costmodelparams_term1 #/(costmodelparams_term1.detach())
                    if(flag_train_memefficient == False):
                        cost_modelparams = cost_modelparams/(stepsize_gradupdate+0.0)
                    else:
                        cost_modelparams = (memefficeint_heads_in_compgraph+0.0)*cost_modelparams/(stepsize_gradupdate+0.0)
                    cost_modelparams.backward()
                    #print("    performed the backprop 3.")
                    if((count_backward_modelparams%stepsize_gradupdate) == 0):
                        optimizer_modelparams.step()
                        optimizer_modelparams.zero_grad()
                        #print("   >>> backprop3.step.")
                    
                    count_updatestomodelparams += 1
                    history_modelparamsloss.append(costmodelparams_term1.detach().cpu().numpy().tolist())
                    #history_celoss.append(celoss_modelprams.detach().cpu().numpy().tolist())
                    
                        
                    
                    #update GPX ====
                    for _ in range(10):
                        gpmodel.update_U()
                    #print("         performed the update to GPX.")
                    
                    #update scheduler of model_params ====
                    if((count_updatestomodelparams % int(len(ds_train)/(batchsize*1.0))) == 0):
                        if(count_updatestomodelparams > 0):
                            #check if the model has to be reverted (the nan issue) ===
                            flag_nanoccured = np.isnan(cost_modelparams.detach().cpu().numpy().tolist())
                            if(flag_nanoccured == False):
                                #update the backup of model and optim
                                torch.save(
                                    gpmodel.state_dict(),
                                    "{}/CheckpointAvoidNan/gpmodel_phase2_w{}.pt"\
                                        .format(path_output_dump, dim_wideoutput)
                                )
                                backup_optimizer_straight = copy.deepcopy(optimizer_straight)
                                backup_optimizer_g = copy.deepcopy(optimizer_g)
                                backup_optimizer_GPY = copy.deepcopy(optimizer_GPY)
                                backup_optimizer_modelparams = copy.deepcopy(optimizer_modelparams)
                            else:
                                #revert-back the model and the optimizer 
                                gpmodel.load_state_dict(
                                        torch.load(
                                           "{}/CheckpointAvoidNan/gpmodel_phase2_w{}.pt".format(
                                               path_output_dump,
                                               dim_wideoutput
                                           )
                                         ),
                                        strict = True
                                     )
                                optimizer_straight = copy.deepcopy(backup_optimizer_straight)
                                optimizer_g = copy.deepcopy(backup_optimizer_g)
                                optimizer_GPY = copy.deepcopy(backup_optimizer_GPY)
                                optimizer_modelparams = copy.deepcopy(backup_optimizer_modelparams)
                                optimizer_straight.zero_grad()
                                optimizer_g.zero_grad()
                                optimizer_GPY.zero_grad()
                                optimizer_modelparams.zero_grad()
                                count_backward_qvn = 1
                                count_backward_qvhatm = 1
                                count_backward_modelparams = 1
                                #optimizer_GPmatchNN.zero_grad()
                                print(">>>>>>>>>>>>>>>>>> NAN OCCURED <<<<<<<<<<<<<<<<<<<.")
                            
                        '''
                        torch.save(
                            gpmodel.state_dict(),
                            "TrainingHistory/Phase2/{}.pt".format(count_updatestomodelparams)
                        )
                        '''
                               
    
    flag_GPXinitialized = True        
            
t_afterepoch_2 = time.time()
print("Round 2 took {} seconds.".format(t_afterepoch_2 - t_beforeepoch_2))

if(epoch>0):
    torch.save(
        gpmodel.state_dict(),
        "{}/round2_epoch_{}.pt".format(path_output_dump, epoch)
    )